# Field updating using Pycap

#### If you want to startup in VSCode:

In [ ]:
# Create and activate virtual environment
! python3 -m venv venv
! source ./venv/bin/activate venv

# Install kernel 
! pip install ipykernel
! python3 -m ipykernel install --user 

#### Install dependencies
- Pycap is the package used to integrate with the REDCap API
- Dotenv is a package used to handle environmental variables

In [ ]:

! pip install pycap
! pip install python-dotenv

## Script Execution

#### Import dependencies

In [85]:
import os
import base64
import sys
from datetime import datetime
from dotenv import load_dotenv
from redcap import Project, RedcapError
load_dotenv()

True

#### Field names

In [90]:
# Default fields
recordPrimaryKey = 'record_id'
recordFileField = 'key'
recordUploadField = 'key_base64'

In [92]:
error = []

if not os.getenv('REDCAP_API_KEYs') or not os.getenv('REDCAP_API_URL'):
    error.append('Environmental variables have not been set. Please update your .env file')
else: 
    try:
        
        # Initialize connection to project
        project = Project(str(os.getenv('REDCAP_API_URL')), str(os.getenv('REDCAP_API_KEY')))
        
        # Grab all record IDs
        records = project.export_records(
            format_type='json', 
            fields=[recordPrimaryKey], 
            raw_or_label='raw'
        )
        print(records)
        newRecords = []
        
        error.append(f'Record 1 has no file uploaded with the field name "key"')
        for record in records: 
            if(int(record[recordPrimaryKey]) < 2):
                # Grab key file from record
                key = project.export_file(
                    record=record[recordPrimaryKey], 
                    field=recordFileField
                )
                
                if key: # encode as base64, convert to string representation and store
                    encoded = base64.b64encode(key[0])
                    newRecord = {
                        recordPrimaryKey: int(record[recordPrimaryKey]),
                        recordUploadField: encoded.decode('ascii')
                    }
                    newRecords.append(newRecord)
                else:
                    error.append(f'Record {record[recordPrimaryKey]} has no file uploaded with the field name {recordFileField}')
        
        project.import_records(newRecords) # import all new records


            
        
    except RedcapError as e:
        error.append(f'REDCap Error : {e}')
    except ValueError as e: 
        error.append(f'Value Error : {e}')
    

if len(error):
    time = datetime.now().strftime('%B %d, %Y %H:%M:%S')
    with open('error.log', 'a') as f:
        f.write(f'\n\n####### Error log for script execution on {time}\n')
        f.write('\n'.join(error))
    